# Scrape Discord

References:
- https://github.com/woctezuma/DiscordScraper
- https://github.com/woctezuma/discord-members-metadata

## Installation

In [ ]:
%cd /content
!git clone https://github.com/woctezuma/DiscordScraper.git --branch patch --single-branch
%cd /content/DiscordScraper
%pip install -qq -r requirements.txt

# https://github.com/dolfies/discord.py-self/pull/618
%pip install -qq git+https://github.com/pc111c/discord.py-self.git@patch-1

## Secrets

In [ ]:
import json

data = {
  "token": "MY_TOKEN", # TODO
  "guild_id": 0, # TODO
  "pfp_format": "png",
  "purge_old_data": False,
  "download_bio": True,
  "download_pfp": False,
  "channel_id": 0 # TODO
}

with open('config.json', 'w') as f:
  json.dump(data, f, indent=2)

## Import data

In [ ]:
GUILD = "MY_GUILD" # TODO
FNAME = f"{GUILD}.txt.zip"

In [ ]:
%cd /content/DiscordScraper
%mkdir -p DataScraped

%cd /content/DiscordScraper/DataScraped
%cp /content/drive/MyDrive/Discord/{FNAME} .
!unzip -qq -o {FNAME}

!ls {GUILD}/*.txt | wc -l

## Run

In [ ]:
%cd /content/DiscordScraper
!python main.py

## Export data

In [ ]:
%cd /content/DiscordScraper/DataScraped
%rm {GUILD}/members.json
!zip -qq -r {FNAME} {GUILD}
%cp {FNAME} /content/drive/MyDrive/Discord/

!ls {GUILD}/*.txt | wc -l

## Aggregate TXT data into a single JSON file

In [ ]:
# Reference: https://github.com/woctezuma/DiscordScraper/blob/49b73e26532e2db8d4b0d550b2e2f3b4533b6ece/internal/utils.py#L161-L164
PATTERN_START = "\nBio: "
PATTERN_END = "\nDiscriminator: #"
DUMMY_BIO = "User doesn't have a bio."

def extract_bio(text, verbose=False):
  bio_start = text.find(PATTERN_START) + len(PATTERN_START)
  bio_end = text.find(PATTERN_END)

  bio = text[bio_start:bio_end].strip()

  if bio == DUMMY_BIO:
    bio = None

  if bio and verbose:
    print(f"{fname}\n{bio}")

  return bio

In [ ]:
from glob import glob
from pathlib import Path

def read_bios_from_text_files(folder_name):
  d = {}

  for fname in glob(f"{folder_name}/*.txt"):
    member_id = Path(fname).stem

    text = Path(fname).read_text(encoding='utf8')
    bio = extract_bio(text)

    if bio:
      d[member_id] = bio

  return d

In [ ]:
import json

bios = read_bios_from_text_files(GUILD)

with open('bios.json', 'w') as f:
  json.dump(bios, f, indent=2, sort_keys=True)